<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/neqsimreaktoro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use of Reaktoro as a unit operation in NeqSim process simulation
In the following example we will integrate Reatoro as an unit operation in a NeqSim process simulation

In [1]:
!pip install neqsim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 4.4 MB/s eta 0:00:00


In [2]:
%%capture
!pip install -q condacolab
import condacolab
condacolab.install_from_url("https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh")
!conda config --remove channels defaults
!conda config --add channels conda-forge
!conda install reaktoro -y

# Implementing a Reaktoro unit operation in NeqSim

In [8]:
import reaktoro as rkt

db = rkt.PhreeqcDatabase("phreeqc.dat")

solution = rkt.AqueousPhase(rkt.speciate("H O Na Cl C Ca"))
solution.set(rkt.ActivityModelPhreeqc(db))

calcite = rkt.MineralPhase("Calcite")

system = rkt.ChemicalSystem(db, solution, calcite)

specs = rkt.EquilibriumSpecs(system)
specs.temperature()
specs.pressure()
specs.fugacity("CO2(g)")

solver = rkt.EquilibriumSolver(specs)

state = rkt.ChemicalState(system)
state.temperature(50.0, "celsius")
state.pressure(10.0, "bar")
state.set("H2O", 1.0, "kg")
state.set("Na+", 1.0, "mol")
state.set("Cl-", 1.0, "mol")
state.set("Calcite", 10, "g")

fCO2g = 0.1  # 0.1 bar

conditions = rkt.EquilibriumConditions(specs)
conditions.temperature(50.0, "celsius")
conditions.pressure(10.0, "bar")
conditions.fugacity("CO2(g)", fCO2g, "bar")

result = solver.solve(state, conditions)
props = rkt.ChemicalProps(state)
print(props)

aprops = rkt.AqueousProps(state)
print(aprops)



+----------------------------------------+--------------+-----------+
| Property                               |        Value |      Unit |
+----------------------------------------+--------------+-----------+
| Temperature                            |     323.1500 |         K |
| Pressure                               |      10.0000 |       bar |
| Volume                                 |   1.0330e-03 |        m3 |
| Gibbs Energy                           |   -5878.1712 |         J |
| Enthalpy                               |    2401.0137 |         J |
| Entropy                                |      25.6203 |       J/K |
| Internal Energy                        |    1368.0387 |         J |
| Helmholtz Energy                       |   -6911.1463 |         J |
| Charge                                 |  -6.9129e-17 |       mol |
| Element Amount:                        |              |           |
| :: H                                   |   1.1101e+02 |       mol |
| :: C              

In [17]:
aprops.pH()[0]

6.712974538117481

In [89]:
from neqsim.process.unitop import unitop
from neqsim import jNeqSim
from jpype import JImplements, JOverride
import reaktoro as rkt
import json

class ReaktoroUnitOperation(unitop):
    def __init__(self):
        super().__init__()
        self.name = ""
        self.inputstream = None
        self.aprops = None
        self.fugacity_co2 = None

    def setInputStream(self, stream):
        self.inputstream = stream

    def calc_reaktoro(self, pressure, temperature , fugacity_co2):
      db = rkt.PhreeqcDatabase("phreeqc.dat")
      solution = rkt.AqueousPhase(rkt.speciate("H O Na Cl C Ca"))
      solution.set(rkt.ActivityModelPhreeqc(db))

      calcite = rkt.MineralPhase("Calcite")

      system = rkt.ChemicalSystem(db, solution, calcite)

      specs = rkt.EquilibriumSpecs(system)
      specs.temperature()
      specs.pressure()
      specs.fugacity("CO2(g)")

      solver = rkt.EquilibriumSolver(specs)

      state = rkt.ChemicalState(system)
      state.temperature(50.0, "celsius")
      state.pressure(10.0, "bar")
      state.set("H2O", 1.0, "kg")
      state.set("Na+", 1.0, "mol")
      state.set("Cl-", 1.0, "mol")
      state.set("Calcite", 10, "g")

      conditions = rkt.EquilibriumConditions(specs)
      conditions.temperature(temperature, "celsius")
      conditions.pressure(pressure, "bar")
      conditions.fugacity("CO2(g)", fugacity_co2, "bar")

      result = solver.solve(state, conditions)
      return rkt.AqueousProps(state)

    @JOverride
    def run(self):
      print("ReaktoroUnitOperation.run()")
      self.fugacity_co2 = self.inputstream.getFluid().getPhase(0).getFugacity('CO2')
      self.aprops = self.calc_reaktoro(self.inputstream.getPressure('bara'), self.inputstream.getTemperature('C'), self.fugacity_co2)

    @JOverride
    def toJson(self):
      data_dict = {
            "name": self.name,
            "pH": self.aprops.pH()[0],
            "fugacity CO2": self.fugacity_co2
      }
      return json.dumps(data_dict)


# Implement the Reaktoro unit operation in a NeqSim simulation
THe unit operation will report the pH of an aqueous phase with some ions in equilibrium with the gas.

In [90]:
from neqsim.thermo import fluid
fluid1 = fluid("srk")  # create a fluid using the SRK-EoS
fluid1.setTemperature(30.0, "C")
fluid1.setPressure(100.0, "bara")
fluid1.addComponent("methane", 0.9)
fluid1.addComponent("CO2", 0.1)
fluid1.setMixingRule(2)

stream1 = jNeqSim.processSimulation.processEquipment.stream.Stream(fluid1)
stream1.setFlowRate(30000, "kg/hr")

uop = ReaktoroUnitOperation()
uop.setName("Reaktoro Unit Operation")
uop.setInputStream(stream1)

example_process = jNeqSim.processSimulation.processSystem.ProcessSystem()
example_process.add(stream1)
example_process.add(uop)

example_process.run()


In [91]:
uop.run() # this should run when process run - but seems not to
json_report= str(jNeqSim.processSimulation.util.report.Report(example_process).generateJsonReport())
output = json.loads(json_report)
print(output['Reaktoro Unit Operation'])

ReaktoroUnitOperation.run()
{'name': 'Reaktoro Unit Operation', 'pH': 5.520539435475417, 'fugacity CO2': 6.970782669382442}
